In [1]:
import logging
from abc import ABC, abstractmethod
from pyspark.sql import DataFrame, SparkSession

In [2]:
class Query(ABC):
    """
    An abstract class for SQL queries
    """
    @abstractmethod
    def query(self, spark: SparkSession, df: DataFrame):
        """
        Execute SQL query on data

        Args:
            spark: Actice SparkSession
        """
        pass

In [3]:
class BestWinRatio(Query):
    """
    Class for SQL query to find the best football team of all time (highest win ratio)
    """
    def query(self, spark: SparkSession, df: DataFrame):
        """
        Finds the best football team
        """
        try:
            df.createOrReplaceTempView("results")
            query_result = spark.sql("""
                                     WITH count_matches AS (
                                        WITH all_matches AS (
                                            SELECT home_team AS team
                                            FROM results
                                            UNION ALL
                                            SELECT away_team AS team 
                                            FROM results
                                        )
                                        SELECT team, 
                                            COUNT(team) AS count_matches
                                        FROM all_matches
                                        GROUP BY team
                                        HAVING COUNT(team) >= 500
                                     ),
                                     winns_count AS (
                                        WITH winner AS(
                                            SELECT home_team, away_team, home_score, away_score, 
                                            CASE
                                                WHEN home_score > away_score THEN home_team
                                                WHEN home_score < away_score THEN away_team
                                                ELSE NULL
                                            END AS winner
                                            FROM results
                                        )
                                        SELECT winner,
                                            COUNT(winner) AS winns_count
                                        FROM winner
                                        GROUP BY winner
                                     ),
                                     win_ratio AS(
                                     SELECT w.winner, w.winns_count, m.team, m.count_matches,
                                     w.winns_count / m.count_matches AS win_ratio
                                     FROM winns_count w
                                     JOIN count_matches m ON w.winner = m.team
                                     )
                                     
                                     SELECT team, win_ratio
                                     FROM win_ratio
                                     ORDER BY win_ratio DESC
                                     LIMIT 10;
                                     """)
            return query_result
        except Exception as e:
            logging.error(f"Error in SQL query (The highest win ratio): {e}")
            raise e

In [4]:
class BestScorer(Query):
    """
    Class for finding the best scorer of all time (the most goals)
    """
    def query(self, spark: SparkSession, df: DataFrame):
        """
        Finds the best scorer of all time
        """
        try:
            df.createOrReplaceTempView("scorers")
            query_result = spark.sql("""
                                     WITH ranked_scores AS(
                                        SELECT scorer, team,
                                            COUNT(scorer) as goal_count,
                                            ROW_NUMBER() OVER (PARTITION BY team ORDER BY COUNT(scorer) DESC) AS rank
                                        FROM scorers
                                        GROUP BY scorer, team
                                     )
                                     
                                     SELECT scorer, goal_count
                                     FROM ranked_scores
                                     WHERE rank = 1 
                                     ORDER BY goal_count DESC
                                     LIMIT 10;
""")
            return query_result
        except Exception as e:
            logging.error(f"Error in SQL query (The best scorer): e")
            raise e

In [5]:
class WorldCupWinner(Query):
    """
    Class for finding every team, which has won Fifa World Cup and count number of trophies
    """
    def query(self, spark: SparkSession, results: DataFrame, shootouts: DataFrame):
        """
        Finds number of Fifa World Cup trophies won by teams

        Args:
            results: DataFrame representing results data
            shootouts: DataFrame representing shootouts data
        """
        try:
            results.createOrReplaceTempView("results")    
            shootouts.createOrReplaceTempView("shootouts")      

            query_result = spark.sql("""
                                     WITH world_cup AS (
                                        SELECT *,
                                            EXTRACT(YEAR FROM date) AS year
                                        FROM results
                                        WHERE tournament = 'FIFA World Cup'
                                     ),
                                     
                                     ranked_matches AS (
                                        SELECT wc.*,
                                            ROW_NUMBER() OVER (PARTITION BY YEAR ORDER BY date DESC) AS match_rank
                                        FROM world_cup wc
                                     ),

                                     final_date AS (
                                        SELECT rm.*
                                        FROM ranked_matches rm
                                        WHERE rm.match_rank = 1
                                     ),

                                     all_matches_final_date AS (
                                        SELECT rm.*
                                        FROM ranked_matches rm 
                                        JOIN final_date fd ON rm.date = fd.date
                                     ),

                                     count_final_matches AS (
                                        SELECT amfd.year,
                                        COUNT(*) AS count_final_matches
                                        FROM all_matches_final_date amfd
                                        GROUP BY amfd.year
                                     ),
                                     
                                     only_final_match AS (
                                        SELECT rm.*,
                                            CASE 
                                                WHEN cfm.count_final_matches = 1 THEN TRUE
                                                WHEN cfm.count_final_matches > 1 THEN FALSE
                                            END AS only_final_match
                                        FROM count_final_matches cfm
                                        JOIN ranked_matches rm ON cfm.year = rm.year
                                     ),
                                     
                                     handle_one_match AS (
                                        SELECT ofm.*
                                        FROM only_final_match ofm
                                        WHERE ofm.only_final_match = TRUE AND ofm.match_rank = 1
                                     ),

                                     find_winner_one_match AS (
                                        SELECT hom.date,
                                            CASE
                                                WHEN hom.home_score > hom.away_score THEN hom.home_team
                                                WHEN hom.home_score < hom.away_score THEN hom.away_team
                                                ELSE NULL
                                            END AS one_match_winner
                                        FROM handle_one_match hom
                                     ),

                                     count_wins_one_match AS (
                                        SELECT fwom.one_match_winner AS winner,
                                            COUNT(fwom.one_match_winner) AS count_wins
                                        FROM find_winner_one_match fwom
                                        GROUP BY fwom.one_match_winner
                                     ),

                                     handle_many_matches AS (
                                     SELECT ofm.*
                                     FROM only_final_match ofm
                                     WHERE ofm.only_final_match = FALSE and OFM.match_rank <= 4
                                     ),

                                     find_winners_many_matches AS (
                                        SELECT hmm.year, hmm.date,
                                            CASE
                                                WHEN hmm.home_score > hmm.away_score THEN hmm.home_team
                                                WHEN hmm.home_score < hmm.away_score THEN hmm.away_team
                                                ELSE NULL
                                            END AS many_matches_winner 
                                        FROM handle_many_matches hmm
                                     ),

                                     count_winner_many_matches AS (
                                        SELECT fwmm.many_matches_winner, fwmm.year,
                                            COUNT(fwmm.many_matches_winner) AS many_matches_count_wins
                                        FROM find_winners_many_matches fwmm
                                        GROUP BY fwmm.year, fwmm.many_matches_winner
                                     ),

                                     rank_winners_many_matches AS (
                                        SELECT cwmm.many_matches_winner, 
                                               cwmm.year,
                                               ROW_NUMBER() OVER (PARTITION BY cwmm.year ORDER BY cwmm.many_matches_count_wins DESC) AS rank_many_matches
                                        FROM count_winner_many_matches cwmm
                                     ),
                                     
                                     find_final_winner_many_matches AS (
                                        SELECT rwmm.many_matches_winner
                                        FROM rank_winners_many_matches rwmm
                                        WHERE rwmm.rank_many_matches = 1
                                     ),

                                     count_wins_many_matches AS (
                                        SELECT ffwmm.many_matches_winner AS winner,
                                            COUNT(ffwmm.many_matches_winner) AS count_wins
                                        FROM find_final_winner_many_matches ffwmm
                                        GROUP BY ffwmm.many_matches_winner
                                     ),

                                     find_nulls_in_shootouts AS (
                                        SELECT s.winner
                                        FROM shootouts s
                                        JOIN find_winner_one_match fwom ON s.date = fwom.date
                                        WHERE fwom.one_match_winner IS NULL
                                        UNION ALL 
                                        SELECT s.winner
                                        FROM shootouts s
                                        JOIN find_winners_many_matches fwmm ON s.date = fwmm.date
                                        WHERE fwmm.many_matches_winner IS NULL
                                     ),

                                     count_wins_in_nulls AS (
                                     SELECT fnis.winner,
                                        COUNT(fnis.winner) AS count_wins
                                     FROM find_nulls_in_shootouts fnis
                                     GROUP BY fnis.winner
                                     ),
                                     
                                     union_all_winners AS (
                                        SELECT cwom.winner, cwom.count_wins
                                        FROM count_wins_one_match cwom
                                        UNION ALL
                                        SELECT cwmm.winner, cwmm.count_wins
                                        FROM count_wins_many_matches cwmm
                                        UNION ALL
                                        SELECT cwin.winner, cwin.count_wins
                                        FROM count_wins_in_nulls cwin
                                        ORDER BY cwom.count_wins DESC
                                     
                                     )
                                     
                                     SELECT uaw.winner,
                                        SUM(uaw.count_wins) AS number_trophies
                                     FROM union_all_winners uaw
                                     WHERE uaw.winner IS NOT NULL
                                     GROUP BY uaw.winner
                                     ORDER BY number_trophies DESC
                                     ;
                                     """)
            return query_result
        except Exception as e:
            logging.error(f"Error in SQL query (Fifa World Cup trophies): {e}")
            raise e